# Simulated Data Manual Test Notebook

This notebook generates and inspects simulated input tables for model evaluation development.

In [ ]:
from pathlib import Path
import sys

# Ensure repository root is importable when notebook runs from notebooks/.
repo_root = Path.cwd().resolve().parent if Path.cwd().name == "notebooks" else Path.cwd().resolve()
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from simulated_data import create_simulated_tables

In [ ]:
model_score, target_store, situation = create_simulated_tables(seed=42)

print("model_score:", model_score.shape)
print("target_store:", target_store.shape)
print("situation:", situation.shape)

In [ ]:
model_score.head()

In [ ]:
target_store.head()

In [ ]:
situation.head()

In [ ]:
print("score min/max:", float(model_score["score"].min()), float(model_score["score"].max()))
print("percentile min/max:", int(model_score["percentile"].min()), int(model_score["percentile"].max()))
print("target min timestamp:", target_store["atsp_event_timestamp"].min())
print("target max timestamp:", target_store["atsp_event_timestamp"].max())

In [ ]:
percentile_summary = (
    model_score.groupby("percentile", as_index=False)
    .agg(
        clients_count=("pt_unified_key", "count"),
        min_score=("score", "min"),
        max_score=("score", "max"),
    )
    .sort_values("percentile")
)

percentile_summary

In [ ]:
assert model_score["pt_unified_key"].between(1, 10000).all()
assert model_score["score"].between(0, 1).all()
assert model_score["percentile"].between(1, 100).all()
assert target_store["pt_unified_key"].nunique() == 1000
assert situation["pt_unified_key"].nunique() == 7000
assert target_store["atsp_event_timestamp"].between("2026-02-01", "2026-02-28 23:59:59").all()
print("All notebook checks passed.")